In [1]:
%env CUDA_VISIBLE_DEVICES=1

env: CUDA_VISIBLE_DEVICES=1


In [2]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import torch
import pickle
from collections import namedtuple, defaultdict
from IPython.display import clear_output
from tqdm.notebook import tqdm, trange, tqdm_notebook
import torch.nn as nn
from radam import RAdam
from torch.optim import Adam
import os
import torch
from torch import nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, random_split
import pytorch_lightning as pl
from pytorch_lightning.loggers import WandbLogger
wandb_logger = WandbLogger()

In [3]:
torch.manual_seed(1233)
device = 'cuda' if torch.cuda.is_available() else 'cpu'
assert device == 'cuda'

BP = 0.0001

mprms = dict(
    epi_length=3000,
    epi_test_length=10_000,
    batch_size = 128,
    hid_size= 512,
    learning_rate = 1e-3,
    epoches = 120,
    alpha = 0.,
    cost = 0.0*BP)


CONFIG = mprms
passport = dict(group = 'Base_1',tags = ['sa'],project_name = 'Zero_Test')


TradingEpisode = namedtuple('TradingEpisode', ['prices', 'features'])
EPS=1e-5 #Now desribed in functions.py

In [4]:
device

'cuda'

In [5]:
f_multiplier = 1_000_000
class MeanReversionSet(torch.utils.data.Dataset):
    def __init__(self, features, prices, epi_len):
        super().__init__()
        self.features = features
        self.prices = prices
        self.epi_len = epi_len
        self.num_ticks = len(features)
    def __getitem__(self, i):
        features = torch.as_tensor(f_multiplier*self.features[i:i+self.epi_len] , dtype = torch.float32)
        prices   = torch.as_tensor(self.prices  [i:i+self.epi_len] , dtype = torch.float32)
        return TradingEpisode(prices = prices, features = features)
    def __len__(self):
        return self.num_ticks - self.epi_len

In [6]:
def calc_m2m(model,  mkte):
    actions = model(mkte.features)

    new_positions = torch.matmul(torch.tensor([-1,0,1.0]).to(device),actions.permute(0,2,1) )

    new_positions[:, -1] = torch.zeros_like(new_positions[:, -1])

    d_positions = F.pad(new_positions[:,1:] - new_positions[:,:-1], (1,0))

    d_cash = -d_positions* mkte.prices
    #print(d_cash.shape)

    cash = d_cash.sum(-1)
    
    m2m = torch.cumsum( d_cash , dim = -1) + new_positions* mkte.prices
    #print(m2m.shape)
    #print(torch.cummax(m2m, dim=-1))
    m2m_drawdown = torch.cummax(m2m, dim=-1)[0]- m2m
    m2m_drawdown = m2m_drawdown.mean(-1).mean()
   # print(m2m_drawdown.shape)
    
    
    m2m_final = cash - torch.mean(model.cost*torch.abs(d_cash)) #####!!!!!!!!! standart: without torch.mean(model.cost*torch.abs(d_cash))
    return m2m_final, cash, new_positions, d_cash, m2m_drawdown, m2m


def calc_episode(model, mkte, add_stat=False):
    stats = defaultdict(list)
    batch_size, n_timestamps, n_features = mkte.features.shape
    zero_state = torch.zeros(batch_size, mprms['hid_size']).to(device)
    initial_hid = (zero_state, zero_state)  # [h, c]
    
#     new_state, actions = model(mkte.features[:,0,:], initial_hid)
    actions = model(mkte.features[:,0,:].to(device))

    positions = torch.zeros(batch_size, requires_grad=True).to(device)
    cash = torch.zeros_like(positions, requires_grad=True).to(device)
    m2m  = torch.zeros_like(positions, requires_grad=True).to(device)
    
    for t in range(1, mprms['epi_length']-1):
#         new_state, actions = model(mkte.features[:,t,:], new_state)
        actions = model(mkte.features[:,t,:].to(device))
        
        if add_stat:
            stats['t'] = t
            stats['actions'].append(actions.cpu().detach().numpy())
            
        new_positions = torch.matmul(torch.tensor([-1,0,1.0]).to(device),actions.T)
        if add_stat:
            stats['positions'].append(new_positions.cpu().detach().numpy())
        d_positions = new_positions - positions
        d_cash = -d_positions* mkte.prices[:,t]
        cash = cash + d_cash
        new_m2m = cash + new_positions * mkte.prices[:,t]

        if add_stat:
            stats['m2m'].append(new_m2m.cpu().detach().numpy())
        
        positions = new_positions
        m2m=new_m2m
        
    return m2m, cash, positions, stats


In [7]:
import wandb
wandb.login()

wandb: Currently logged in as: s_artamonov (use `wandb login --relogin` to force relogin)


True

In [8]:
def train_log(loss, sharpe,tau,example_ct, epoch):
    loss = float(loss)
    # where the magic happens
    wandb.log({"epoch": epoch, "train/loss": loss,'train/sharpe' : sharpe}, step=example_ct)

def train_val_log(sharpe, mean_R,example_ct, epoch):
    # where the magic happens
    wandb.log({"epoch": epoch, 'val/sharpe' : sharpe,'val/mean_R' : mean_R}, step=example_ct)

In [9]:
class PrtfNNLin(pl.LightningModule):

    def __init__(self,num_features, hid_size, config, mkte,mkte_test, mkt_loader, tau=0.5):
        super().__init__()
        self.L1 = nn.Linear(num_features, hid_size)
        self.L1A = nn.Tanh()
        self.hid_to_actions = nn.Linear(hid_size, 3)
        self.tau = tau
        self.hard = False
        
        self.mkte = mkte
        self.mkt_loader = mkt_loader
        self.mkte_test = mkte_test
        
        self.n_iter = config['epoches']
        self.alpha = config['alpha']
        self.lr = config['learning_rate']
        self.cost = config['cost']

    def forward(self, inp_features):
        inp_features = inp_features.to(device)
        l2 = self.L1(inp_features) 
        #print(l2.shape)        
        l2a = self.L1A(l2)
        action_seq = self.hid_to_actions( l2a )
        final_actions = torch.nn.functional.gumbel_softmax(action_seq, hard=self.hard, tau=self.tau)
        return final_actions

    def training_step(self, batch, batch_idx):
        mkte = batch
        mkte = TradingEpisode(*map(lambda x: x.to(device), mkte))
        m2m_final, cash, new_positions, d_cash, m2m_drawdown, M2M = calc_m2m(self, mkte)
        mean_ret = m2m_final.mean()
               # print('M2M.shape: {} \nmean_ret: {}'.format(m2m.shape,mean_ret))
        sharpe = mean_ret / (m2m_final.std() + EPS)
                #print(m2m.shape)
                #print(sharpe.shape)
                #loss = -sharpe - alpha*m2m_drawdown     Very Stupid))
        #print('len(d_cash) = ',d_cash.shape)
        loss = -sharpe + self.alpha*m2m_drawdown +torch.mean(self.cost*torch.abs(d_cash)) #self.cost*torch.abs(d_cash[-1]) #torch.sum(self.cost*torch.abs(d_cash))
            
        # Logging to TensorBoard by default
        self.logger.experiment.log({'train_loss': loss.item(),'train_sharpe':sharpe.item()})
        
        #m2m_final, cash, new_positions, d_cash, m2m_drawdown, m2m
        with torch.no_grad():
            m2m, cash, positions,_,_,_ = calc_m2m(self, self.mkte_test)
        R = np.stack(m2m.cpu().detach().numpy()).reshape(-1)
        mean_R = np.mean(R)
        sharpe_val = mean_R / (np.std(R) + EPS)
        
        self.logger.experiment.log({'val/mean_R': mean_R,'val/sharpe':sharpe_val})
            
                

        #self.logger.experiment.log()
        return loss

    def configure_optimizers(self):
        optimizer = RAdam(self.parameters(), lr=self.lr)
        return optimizer

In [10]:
def setup_experiment(config, comp):
    # Make the data
    
    
    features = pd.DataFrame(dict(px = comp)).assign(
    dpx1 = lambda x: x.px - x.px.ewm(64).mean(),
    dpx2 = lambda x: x.px.ewm(64).mean() - x.px.ewm(128).mean(),
    dpx3 = lambda x: x.px.ewm(128).mean() - x.px.ewm(256).mean(),
    dpx4 = lambda x: x.px.ewm(256).mean() - x.px.ewm(512).mean())

    
    split_idx = int(comp.shape[0]*0.7)
    train_comp = comp[:split_idx]
    test_comp = comp[split_idx:]
    
    train_features = pd.DataFrame(dict(px = train_comp)).assign(
    dpx1 = lambda x: x.px - x.px.ewm(64).mean(),
    dpx2 = lambda x: x.px.ewm(64).mean() - x.px.ewm(128).mean(),
    dpx3 = lambda x: x.px.ewm(128).mean() - x.px.ewm(256).mean(),
    dpx4 = lambda x: x.px.ewm(256).mean() - x.px.ewm(512).mean())

    test_features = features[split_idx:]
    
    
    
   # train, test = get_data(train=True), get_data(train=False)
    #train_loader = make_loader(train, batch_size=config.batch_size)
    #test_loader = make_loader(test, batch_size=config.batch_size)
    
    f_train_values = train_features.filter(like = 'dp').values
    f_test_values = test_features.filter(like = 'dp').values
    
    dataset = MeanReversionSet(f_train_values, train_features['px'].values, epi_len = mprms['epi_length'])
    mkt_loader = torch.utils.data.DataLoader(dataset, batch_size=mprms['batch_size'], shuffle=True, num_workers=4)
    n_features = f_train_values.shape[1]

    testset = MeanReversionSet(f_test_values, test_features['px'].values, epi_len = mprms['epi_test_length'])
    mkt_tst_loader = torch.utils.data.DataLoader(testset, batch_size=1, shuffle=False, num_workers=1)
    n_test_features = f_test_values.shape[1]
    


    # Make the model

   

    mkte = next(iter(mkt_loader))
    mkte = TradingEpisode(*map(lambda x: x.to(device), mkte))
    zero_state = torch.zeros(config['batch_size'], config['hid_size']).to(device)
    initial_hid = (zero_state, zero_state)  # [h, c]
    # res = model(mkte.features[:,0,:], initial_hid)
    #res = model(mkte.features[:,0,:])
    
    mkte_test = next(iter(mkt_tst_loader))
    mkte_test = TradingEpisode(*map(lambda x: x.to(device), mkte_test))
    zero_state_tst = torch.zeros(1, mprms['hid_size']).to(device)
    initial_hid_tst = (zero_state_tst, zero_state_tst)  # [h, c]  
    
    model = PrtfNNLin(n_features,mprms['hid_size'],config, mkte,mkte_test, mkt_loader).to(device)
    
    return model, dataset, testset, mkte, mkte_test, mkt_loader, mkt_tst_loader

In [11]:
with open('comp.pck', 'rb') as f:
    comp = pickle.load(f)
comp = comp[:70000]

In [12]:
def test(model, mkte_test):
    with torch.no_grad():
        m2m, cash, positions, stat = calc_episode(model, mkte_test, True)
    RW = m2m.mean() / (m2m.std() + EPS)
    plt.plot( np.stack(stat['m2m']).T[0,:])
    
def model_pipeline(hyperparameters, comp):
    # tell wandb to get started
      with wandb.init(project=passport['project_name'], group=passport['group'], tags=passport['tags'], config=hyperparameters, save_code=False):    
#    with wandb.init(config=hyperparameters):

    # access all HPs through wandb.config, so logging matches execution!
            config = wandb.config

      # make the model, data, and optimization problem
            model, dataset, testset, mkte, mkte_test, mkt_loader, mkt_tst_loader = setup_experiment(CONFIG, comp)
            print(model)
            trainer = pl.Trainer(gpus=1,logger=wandb_logger,max_epochs=hyperparameters['epoches'])
            with tqdm(mkt_loader) as mkt_loader:
                trainer.fit(model,mkt_loader)
      # and use them to train the model


      # and test its final performance
           # test(model, mkte_test)

      return model,mkte_test

In [15]:
MODELS = []
MKTE_TESTS = []

ALPHAS = [0.05,0.1,0.15]
COSTS = [0.1,0.5]

In [ ]:
for alpha in ALPHAS:
    for cost in COSTS:
        mprms = dict(
                        epi_length=3000,
                        epi_test_length=10_000,
                        batch_size = 128,
                        hid_size= 512,
                        learning_rate = 1e-3,
                        epoches = 120,
                        alpha = alpha,
                        cost = cost*BP)


        CONF = mprms
        model,mkte_test = model_pipeline(CONF,comp)
        
        MODELS.append(model)
        MKTE_TESTS.append(mkte_test)
        
        

GPU available: True, used: True
TPU available: None, using: 0 TPU cores


PrtfNNLin(
  (L1): Linear(in_features=4, out_features=512, bias=True)
  (L1A): Tanh()
  (hid_to_actions): Linear(in_features=512, out_features=3, bias=True)
)



  | Name           | Type   | Params
------------------------------------------
0 | L1             | Linear | 2.6 K 
1 | L1A            | Tanh   | 0     
2 | hid_to_actions | Linear | 1.5 K 
------------------------------------------
4.1 K     Trainable params
0         Non-trainable params
4.1 K     Total params
0.016     Total estimated model params size (MB)


/home/Serega200010/T-Digital_Project/radam.py:58: UserWarning: This overload of addcmul_ is deprecated:
	addcmul_(Number value, Tensor tensor1, Tensor tensor2)
Consider using one of the following signatures instead:
	addcmul_(Tensor tensor1, Tensor tensor2, *, Number value) (Triggered internally at  /opt/conda/conda-bld/pytorch_1607370172916/work/torch/csrc/utils/python_arg_parser.cpp:882.)
  exp_avg_sq.mul_(beta2).addcmul_(1 - beta2, grad, grad)


IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

IOPub message rate exceed

Problem at: <ipython-input-12-b702c3e9203e> 9 model_pipeline


KeyboardInterrupt: 

In [ ]:
test(model.to(device),mkte_test)

GPU available: True, used: True
TPU available: None, using: 0 TPU cores


NameError: name 'mkt_loader' is not defined

PrtfNNLin(
  (L1): Linear(in_features=4, out_features=512, bias=True)
  (L1A): Tanh()
  (hid_to_actions): Linear(in_features=512, out_features=3, bias=True)
)